In [ ]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/IndicCorpV2", "indiccorp_v2", split="hin_Deva",streaming=True)

In [2]:
dataset

IterableDataset({
    features: Unknown,
    num_shards: 3
})

In [3]:
for i, example in enumerate(dataset):
    print(example)
    if i == 4:
        break

{'text': 'लोगों को बिलों संबंधी सुविधा देना ही उनका काम'}
{'text': ''}
{'text': 'इनेलो 1987 में उस वक्त ऐसे ही दोराहे पर खड़ी थी, जब पूर्व उपप्रधानमंत्री देवीलाल ने अपने पुत्र ओमप्रकाश चौटाला को अपना राजनीतिक उत्तराधिकारी घोषित किया था। हालांकि तब पार्टी पर देवीलाल की मजबूत पकड़ के चलते पार्टी टूटने से बच गई थी। 1989 में देवीलाल केन्द्र की राजनीति में सक्रिय हो गए थे और उनके उपप्रधानमंत्री बनने के पश्चात् उनके तीन बेटों जगदीश सिंह, रणजीत सिंह और ओमप्रकाश चौटाला में से रणजीत और ओमप्रकाश के बीच हरियाणा में उनकी राजनीतिक विरासत को लेकर जंग शुरू हो गई थी। उन परिस्थितियों में देवीलाल ने कड़ा निर्णय लेते हुए पार्टी की बागडोर ओमप्रकाश चौटाला के हवाले कर दी थी, जिसके बाद रणजीत की बगावत का असर पार्टी, संगठन और उनकी सरकार पर भी पड़ा था। उस समय रणजीत की नाराजगी के चलते उनके समर्थन में कई कैबिनेट मंत्रियों ने इस्तीफे दे दिए थे किन्तु तब पार्टी सुप्रीमो चौ. देवीलाल की हरियाणा की जनता पर इतनी मजबूत पकड़ थी कि ओमप्रकाश चौटाला को उत्तराधिकारी बनाने के उनके फैसले का जनता के बीच कोई खास विरोध नहीं हुआ थ

In [4]:
import re
def sentence_tokenizer(paragraph):
  pattern='[\u0964!?.][^0-9.0-9]'
  sentences=re.split(pattern,paragraph)
  ends=re.findall(pattern,paragraph)
  res=[]
  for sentence in sentences:
    if ends==[]:
      break
    end=ends.pop(0)
    res.append(sentence.strip() + end)
  return res


def word_tokenizer(sentence):
    url_pattern = r'https?://(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}'
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    # Phone numbers (with optional +91 or +९१)
    phone_pattern = r'(?:\+?[0-9\u0966-\u096F]{1,3}[\s-]?)?[0-9\u0966-\u096F]{10}'

    # Time format: 10:30 or १०:३०
    time_pattern = r'[0-9\u0966-\u096F]{1,2}:[0-9\u0966-\u096F]{2}'

    # Decimal and whole numbers
    latin_number = r'[0-9]+(?:\.[0-9]+)?'
    devnag_number = r'[\u0966-\u096F]+(?:\.[\u0966-\u096F]+)?'

    # Punctuation
    punctuation = r'[!?\.\u0964\,"\'\+-]'

    # Hindi / Devanagari words
    hindi_word = fr'[\u0900-\u0963\u0965-\u096F]+[^!?\.\u0964\,"\'\+-]'
    english_word = r'[a-zA-Z]+'


    # Emojis (basic unicode emoji range)
    emoji_pattern = r'[\U0001F300-\U0001FAFF\U00002700-\U000027BF]'

    # Final combined pattern
    pattern = fr'{email_pattern}|{url_pattern}|{phone_pattern}|{time_pattern}|{latin_number}|{devnag_number}|{punctuation}|{hindi_word}|{emoji_pattern}|{english_word}'

    return re.findall(pattern, sentence)





In [5]:
import os
from tqdm import tqdm

output_file = "tokenized_hindi.txt"
max_size_bytes = 2 * 1024 * 1024 * 1024  # 2GB

with open(output_file, "w", encoding="utf-8") as fout:
    for row in tqdm(dataset, desc="Tokenizing dataset"):
        text = row['text']
        # Sentence tokenize 
        sentences = sentence_tokenizer(text)
        for sent in sentences:
            tokens = word_tokenizer(sent)
            fout.write(" ".join(tokens) + "\n")
            fout.flush()  # Ensures os.path.getsize is accurate
            if os.path.getsize(output_file) >= max_size_bytes:
                print("File reached 2GB. Stopping...")
                break
        if os.path.getsize(output_file) >= max_size_bytes:
            break


Tokenizing dataset: 2589616it [33:16, 1297.30it/s]


KeyboardInterrupt: 

In [9]:
def split_file_by_size(input_path, output_path, max_bytes):
    read_bytes = 0
    chunk_size = 1024 * 1024  # 1MB
    with open(input_path, 'rb') as fin, open(output_path, 'wb') as fout:
        while read_bytes < max_bytes:
            to_read = min(chunk_size, max_bytes - read_bytes)
            data = fin.read(to_read)
            if not data:
                break
            fout.write(data)
            read_bytes += len(data)

# Usage example:
GB = 1024 ** 3
split_file_by_size('tokenized_hindi.txt', 'tokenized_hindi_10gb.txt', max_bytes=10 * GB)
